<a href="https://colab.research.google.com/github/kyaoNK/bert_asag/blob/main/BERT_ASAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
%%capture
!sudo apt -y update
# !sudo apt -y -q install libeigen3-dev libprotobuf-dev protobuf-c-compiler

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%%capture
# !pip install -qU pytorch-lightning lightning-bolts 
# !pip install -q   tqdm
# !pip install -qU torch
!pip install -qU torchinfo
# !pip install -qU torchtext
# !pip install -qU torchvision
# !pip install -qU torchaudio
!pip install -q   datasets
# !pip install -q   accelerate
!pip install -q   fugashi
!pip install -q   ipadic
# !pip install -q   unidic-lite
!pip install -qU transformers
!pip install -q   sentencepiece
# !pip install -U   sentence-transformers
!pip install natsort

In [4]:
import sys
import os
sys.path.append('/content/drive/MyDrive/Colab Notebooks/my-modules')
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

# Datasets

In [5]:
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

In [6]:
import os
import numpy as np
import pandas as pd
import glob
from natsort import natsorted
import torch
from sklearn.metrics import accuracy_score , f1_score , classification_report , ConfusionMatrixDisplay , confusion_matrix
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
from torchinfo import summary
from datasets import load_dataset , concatenate_datasets

In [7]:
DATASET_DIR = '/content/drive/MyDrive/Colab Notebooks/SeniorThesis/DATASETS/'
JDATASET_DIR = DATASET_DIR + 'OUTPUT/'
TDATASET_DIR = DATASET_DIR + 'OUTPUT/tutoring/'

j_datasets = load_dataset(
    path='csv',
    data_files={
        'jglue' : JDATASET_DIR + 'jglue.csv',
        'jsick' : JDATASET_DIR + 'jsick.csv'
    },
)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating jglue split: 0 examples [00:00, ? examples/s]

Generating jsick split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-befc902b745bf880/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
datafiles_dict = dict()
filepaths = natsorted(glob.glob(TDATASET_DIR + '*.csv' ))
for fp in filepaths:
    filename = os.path.basename(fp)
    key = filename[:-4]
    datafiles_dict[key] = fp
print(datafiles_dict)

t_datasets = load_dataset(
    path='csv',
    data_files=datafiles_dict
)

{'a_ca_0_train': '/content/drive/MyDrive/Colab Notebooks/SeniorThesis/DATASETS/OUTPUT/tutoring/a_ca_0_train.csv', 'a_ca_0_valid': '/content/drive/MyDrive/Colab Notebooks/SeniorThesis/DATASETS/OUTPUT/tutoring/a_ca_0_valid.csv', 'a_ca_1_train': '/content/drive/MyDrive/Colab Notebooks/SeniorThesis/DATASETS/OUTPUT/tutoring/a_ca_1_train.csv', 'a_ca_1_valid': '/content/drive/MyDrive/Colab Notebooks/SeniorThesis/DATASETS/OUTPUT/tutoring/a_ca_1_valid.csv', 'a_ca_2_train': '/content/drive/MyDrive/Colab Notebooks/SeniorThesis/DATASETS/OUTPUT/tutoring/a_ca_2_train.csv', 'a_ca_2_valid': '/content/drive/MyDrive/Colab Notebooks/SeniorThesis/DATASETS/OUTPUT/tutoring/a_ca_2_valid.csv', 'a_ca_3_train': '/content/drive/MyDrive/Colab Notebooks/SeniorThesis/DATASETS/OUTPUT/tutoring/a_ca_3_train.csv', 'a_ca_3_valid': '/content/drive/MyDrive/Colab Notebooks/SeniorThesis/DATASETS/OUTPUT/tutoring/a_ca_3_valid.csv', 'a_ca_4_train': '/content/drive/MyDrive/Colab Notebooks/SeniorThesis/DATASETS/OUTPUT/tutoring/a

Computing checksums:  15%|#5        | 6/40 [00:05<00:29,  1.16it/s]

Extracting data files:   0%|          | 0/40 [00:00<?, ?it/s]

Generating a_ca_0_train split: 0 examples [00:00, ? examples/s]

Generating a_ca_0_valid split: 0 examples [00:00, ? examples/s]

...

Generating q_qa_3_valid split: 0 examples [00:00, ? examples/s]

Generating q_qa_4_train split: 0 examples [00:00, ? examples/s]

Generating q_qa_4_valid split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-dd59edf5c7e569f0/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/40 [00:00<?, ?it/s]

In [9]:
j_datasets

DatasetDict({
    jglue: Dataset({
        features: ['sentenceA', 'sentenceB', 'label'],
        num_rows: 8907
    })
    jsick: Dataset({
        features: ['sentenceA', 'sentenceB', 'label'],
        num_rows: 9927
    })
})

In [10]:
t_datasets

DatasetDict({
    a_ca_0_train: Dataset({
        features: ['sentenceA', 'sentenceB', 'label'],
        num_rows: 359
    })
    a_ca_0_valid: Dataset({
        features: ['sentenceA', 'sentenceB', 'label'],
        num_rows: 91
    })
    a_ca_1_train: Dataset({
        features: ['sentenceA', 'sentenceB', 'label'],
        num_rows: 354
    })
    a_ca_1_valid: Dataset({
        features: ['sentenceA', 'sentenceB', 'label'],
        num_rows: 96
    })
    a_ca_2_train: Dataset({
        features: ['sentenceA', 'sentenceB', 'label'],
        num_rows: 363
    })
    a_ca_2_valid: Dataset({
        features: ['sentenceA', 'sentenceB', 'label'],
        num_rows: 87
    })
    a_ca_3_train: Dataset({
        features: ['sentenceA', 'sentenceB', 'label'],
        num_rows: 366
    })
    a_ca_3_valid: Dataset({
        features: ['sentenceA', 'sentenceB', 'label'],
        num_rows: 84
    })
    a_ca_4_train: Dataset({
        features: ['sentenceA', 'sentenceB', 'label'],
        num

In [11]:
MODEL_CKPT = "cl-tohoku/bert-base-japanese-whole-word-masking"
tokenizer = AutoTokenizer.from_pretrained(MODEL_CKPT)

def tokenize(x):
    return tokenizer( x['sentenceA'] , x['sentenceB'] , padding=True , truncation=True)

tokenized_j_datasets = j_datasets.map(tokenize , batched=True , batch_size=16, remove_columns=['sentenceA', 'sentenceB'])
tokenized_t_datasets = t_datasets.map(tokenize , batched=True , batch_size=16, remove_columns=['sentenceA', 'sentenceB'])

  0%|          | 0/557 [00:00<?, ?ba/s]

  0%|          | 0/621 [00:00<?, ?ba/s]

  0%|          | 0/23 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/23 [00:00<?, ?ba/s]

...

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/23 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/23 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

In [12]:
tokenized_j_datasets

DatasetDict({
    jglue: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 8907
    })
    jsick: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9927
    })
})

In [13]:
tokenized_t_datasets

DatasetDict({
    a_ca_0_train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 359
    })
    a_ca_0_valid: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 91
    })
    a_ca_1_train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 354
    })
    a_ca_1_valid: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 96
    })
    a_ca_2_train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 363
    })
    a_ca_2_valid: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 87
    })
    a_ca_3_train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 366
    })
    a_ca_3_valid: Dataset({
        features

# Finetuning

In [14]:
num_labels = 7
model = (AutoModelForSequenceClassification.from_pretrained(MODEL_CKPT, num_labels=num_labels).to(device))
summary(model, depth=3)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

Layer (type:depth-idx)                                  Param #
BertForSequenceClassification                           --
├─BertModel: 1-1                                        --
│    └─BertEmbeddings: 2-1                              --
│    │    └─Embedding: 3-1                              24,576,000
│    │    └─Embedding: 3-2                              393,216
│    │    └─Embedding: 3-3                              1,536
│    │    └─LayerNorm: 3-4                              1,536
│    │    └─Dropout: 3-5                                --
│    └─BertEncoder: 2-2                                 --
│    │    └─ModuleList: 3-6                             85,054,464
│    └─BertPooler: 2-3                                  --
│    │    └─Linear: 3-7                                 590,592
│    │    └─Tanh: 3-8                                   --
├─Dropout: 1-2                                          --
├─Linear: 1-3                                           5,383
Total params: 11

In [15]:
def compute_metrics(pred) :
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels , preds , average='macro')
    acc = accuracy_score(labels , preds)
    return {'accuracy' : acc , 'f1' : f1}

# training args
batch_size = 1

training_args = TrainingArguments(
    output_dir="/content/",
    num_train_epochs=5,
    # learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    disable_tqdm=False,
    logging_strategy="epoch",
    push_to_hub=False,
    load_best_model_at_end=True,
    log_level="error",
    save_total_limit=10
)

In [16]:
N_KFOLD = 5

# for j in [ 'no' , 'jg' , 'js' , 'jgs'] :
for j in [ 'no' ] :
    if j == 'no' :
        j_dataset = None
    elif j == 'jg' :
        j_dataset = tokenized_j_datasets['jglue']
    elif j == 'js' :
        j_dataset = tokenized_j_datasets['jsick']
    elif j == 'jgs' :
        j_dataset = concatenate_datasets( [ tokenized_j_datasets['jglue'] , tokenized_j_datasets['jsick'] ] )
    
    for d in [ 'ca' , 'qa' ] :
        # for t in [ 'a' , 'q' ]:
        for t in [ 'q' , 'a' ]:
            for idx in range(N_KFOLD):
                data_name = t + '_' + d + '_' + str(idx)
                train_name = data_name + '_train'
                valid_name = data_name + '_valid'
                train_dataset = tokenized_t_datasets[train_name]
                if j_dataset != None :
                    train_dataset = concatenate_datasets( [ train_dataset , j_dataset ] )
                valid_dataset = tokenized_t_datasets[valid_name]
                print(f'{j}, train: {train_name}={len(train_dataset)} , valid: {valid_name}={len(valid_dataset)}')

                trainer = Trainer(
                    model=model,
                    args=training_args,
                    compute_metrics=compute_metrics,
                    train_dataset=train_dataset,
                    eval_dataset=valid_dataset,
                    tokenizer=tokenizer
                )
                trainer.train()

                preds_output = trainer.predict(tokenized_t_datasets[valid_name])
                y_pred = np.argmax(preds_output.predictions, axis=1)
                y_true = np.array(tokenized_t_datasets[valid_name]['label']) # label
                if idx == 0 :
                    preds = y_pred
                    trues = y_true
                else :
                    preds = np.append(preds, y_pred)
                    trues = np.append(trues, y_true)

            cr = classification_report(trues , preds)
            print(f'\n================= {j} , {d} , {t} ===================\n')
            print(cr)
            print(f'===============================================\n')


no, train: q_ca_0_train=360 , valid: q_ca_0_valid=90


/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.651300,2.479416,0.422222,0.111772
2,1.525100,1.959079,0.544444,0.249599
3,1.157400,1.680876,0.588889,0.334833
4,0.793000,1.789277,0.544444,0.267511
5,0.557100,1.982926,0.555556,0.417440


no, train: q_ca_1_train=360 , valid: q_ca_1_valid=90


/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


...

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.398300,0.631220,0.890110,0.836032
2,0.411400,0.570957,0.912088,0.835535
3,0.228300,0.546338,0.923077,0.879261
4,0.115900,0.451456,0.945055,0.886538
5,0.060500,0.492340,0.934066,0.861273


no, train: a_ca_1_train=354 , valid: a_ca_1_valid=96


/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.412700,0.521539,0.927083,0.917516
2,0.507300,0.779237,0.895833,0.905381


KeyboardInterrupt: ignored